In [0]:

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length

In [0]:
# Read from the Bronze Table 
df = spark.table("workspace.bronze.crm_sales_details")
#df.display()

In [0]:
%sql
select sls_sales, sls_quantity, sls_price  from workspace.bronze.crm_sales_details
where sls_quantity > 1

In [0]:
# Trim the white spaces
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))
  

In [0]:
%sql
Describe workspace.bronze.crm_sales_details

Sales and Price Correction

In [0]:
# there is inconsistency in the sales price and the sales quantity , which needs to fix
df = (
    df
    .withColumn(
        "sls_price",
        F.when(
            (col("sls_price").isNull()) | (col("sls_price") <= 0),
            F.when(
                col("sls_quantity") != 0,
                col("sls_sales") / col("sls_quantity")
            ).otherwise(None)
        ).otherwise(col("sls_price"))
    )
)



Change the data format, it is in Int data type

In [0]:
df = (
    df
    .withColumn(
        "sls_order_dt",
        F.when(
            (col("sls_order_dt") == 0) | (length(col("sls_order_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_order_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_ship_dt",
        F.when(
            (col("sls_ship_dt") == 0) | (length(col("sls_ship_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_due_dt",
        F.when(
            (col("sls_due_dt") == 0) | (length(col("sls_due_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_due_dt").cast("string"), "yyyyMMdd"))
    )
)

In [0]:
df.show()

In [0]:
# Rename the columns
Rename_map = {
    "sls_ord_num": "order_number",
    "sls_prd_key": "product_number",
    "sls_cust_id": "customer_id",
    "sls_order_dt": "order_date",
    "sls_ship_dt": "ship_date",
    "sls_due_dt": "due_date",
    "sls_sales": "sales_amount",
    "sls_quantity": "quantity",
    "sls_price": "price"
}
for old_name, new_name in Rename_map.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
# sanity check 
df.limit(15).display()


In [0]:
# write to the Silver table
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_sales")

In [0]:
%sql
select * from workspace.silver.crm_sales